<a href="https://colab.research.google.com/github/czeacach/fashion_designers/blob/main/wdt_spatialisation_lieux_d'%C3%A9tudes_production.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatialisation des naissances: production des données

In [ ]:
### Noter qu'il faudra avoir installé toutes les librairies nécessaires
# dans l'environnement conda utilisé pour exécuter ce carnet
import pandas as pd
import geopandas as gpd
#from shapely.geometry import LineString
from geopandas.tools import sjoin
#import networkx as nx
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
#import seaborn as sns
import json
import IPython

In [ ]:
!pip install geodatasets
!pip install itables
!pip install sparqlwrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
from geodatasets import get_path
from itables import init_notebook_mode, show

In [ ]:
### Librairies déjà installées avec Python
import pprint
import sqlite3 as sql
import sys
from importlib import reload

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab_notebooks')
import sparql_functions as spqf

## Production des données

In [ ]:
## define SPARQL enpoint
endpoint = "https://query.wikidata.org/sparql"

In [ ]:
query = """
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>

SELECT DISTINCT ?item ?itemLabel (?year AS ?birthYear) ?organisation ?organisationLabel ?coordinates ?country ?countryLabel
WHERE {
  {
    { ?item wdt:P106 wd:Q3501317 } # occupation is fashion designer.
    UNION
    { ?item wdt:P101 wd:Q29583 } # field of work is fashion design
    ?item wdt:P31 wd:Q5; # Any instance of a human.
          wdt:P569 ?birthDate.
          ?item wdt:P69 ?organisation. # insitution d'études
     OPTIONAL { ?organisation wdt:P625 ?coordinates}
                  {?organisation wdt:P17 ?country}

    BIND(REPLACE(str(?birthDate), "(.*)([0-9]{4})(.*)", "$2") AS ?year)
    FILTER(xsd:integer(?year) > 1800 && xsd:integer(?year) < 2001)

    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
   }
  }
  ORDER BY ?year ?itemLabel

  """

In [ ]:
### Executer la requête avec les fonctions de la librairie locale
qr = spqf.get_json_sparql_result(endpoint,query)

In [ ]:
r = [l for l in spqf.sparql_result_to_list(qr)]
print(len(r))
r[:3]

2119


[['http://www.wikidata.org/entity/Q1255520',
  'Alfred Isidore Römer',
  '1832',
  'http://www.wikidata.org/entity/Q414052',
  'Academy of Fine Arts, Munich',
  'Point(11.580312391 48.15319592)',
  'http://www.wikidata.org/entity/Q183',
  'Germany'],
 ['http://www.wikidata.org/entity/Q1255520',
  'Alfred Isidore Römer',
  '1832',
  'http://www.wikidata.org/entity/Q658192',
  'Vilnius University',
  'Point(25.287222222 54.6825)',
  'http://www.wikidata.org/entity/Q37',
  'Lithuania'],
 ['http://www.wikidata.org/entity/Q3177164',
  'Jenny Sacerdote',
  '1868',
  'http://www.wikidata.org/entity/Q481762',
  'École normale supérieure de jeunes filles',
  'Point(2.33131 48.8224)',
  'http://www.wikidata.org/entity/Q142',
  'France']]

In [ ]:
### Exporter les résultats en csv et créer une nouvelle table wdt_place_study pour y importer les données

### Nettoyage des données

Requêtes SQL et corrections effectuées directement dans SQlite, à l'aide des requêtes suivants:

**Requête pour calculer le nombre d'établissement sans nom**
```
SELECT COUNT(*) AS organisationLabe_without_name
FROM wdt_place_study
WHERE organisationLabel LIKE 'Q%' AND
      LENGTH(organisationLabel) > 1 AND
      organisationLabel GLOB 'Q[0-9]*';
```
14

J'ai rempli lorsque possible les noms des organisations qui n'en possédaient pas en effectuant quelques recherches

**Requête pour calculer le nombre d'établissement sans coordonnées géographiques**

```
SELECT COUNT(*) AS number_of_individuals_without_coordinates
FROM wdt_place_study
WHERE coordinates IS NULL OR coordinates = '';
```
137


Parmi les 137 lignes  indiquer les coordonnées des lieux qui avaient jusqu’à 3 individus comme effectif, j'ai sélectionné les établissements avec les effectifs les plus importants et j'ai indiqué les coordonnées manquantes pour ceux dont l'effectif était égal ou supérieur à 3.



**Requête pour sélectionner et classer par ordre descendant les établissements sans coordonnées géographiques**

```
SELECT organisationLabel, COUNT(*) AS count
FROM wdt_place_study
WHERE coordinates IS NULL OR coordinates = ''
GROUP BY organisationLabel
ORDER BY count DESC;

```






